In [18]:
%load_ext Cython
%load_ext line_profiler

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [19]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, e, m_p
from scipy.signal import fftconvolve

%matplotlib tk
import seaborn as sns
sns.set_context('notebook', font_scale=1.5,
                rc={'lines.markeredgewidth': 1})
sns.set_style('darkgrid', {
        'axes.linewidth': 2,
        'legend.fancybox': True})

In [20]:
from SPS import SPS

Here we can play with **tunes** and **n_segments** - really the combination of the two, plus the type of non-linearity, determine the resonances excited...

In [61]:
# =====================================
# Input parameters
n_segments = 3
Qx, Qy = 20.13, 20.18

n_turns = 2048
n_sigmas = 6
n_macroparticles = 400
sx, sy = 0.5e-3, 0.5e-3

# =====================================
# Build machine and bunch
x = np.zeros((n_turns, n_macroparticles))
xp = np.zeros((n_turns, n_macroparticles))
y = np.zeros((n_turns, n_macroparticles))
yp = np.zeros((n_turns, n_macroparticles))

C = np.pi*50*44
sps = SPS(machine_configuration='Q20-flattop', RF_at='end_of_transverse', optics_mode='non-smooth',
          name=['s{:d}'.format(i) for i in xrange(n_segments+1)],
          s=[i*C/n_segments for i in xrange(n_segments+1)],
          accQ_x=[i*Qx/n_segments for i in xrange(n_segments+1)],
          accQ_y=[i*Qy/n_segments for i in xrange(n_segments+1)],
          beta_x=np.array([50, 100, 10, 50]), beta_y=np.array([50, 10, 100, 50]),
          alpha_x=np.zeros(n_segments+1), alpha_y=np.zeros(n_segments+1)
          )
sps.one_turn_map.pop(-1)
ax = sps.transverse_map.alpha_x[0]
ay = sps.transverse_map.alpha_y[0]
bx = sps.transverse_map.beta_x[0]
by = sps.transverse_map.beta_y[0]

Synchrotron init. From kwargs: RF_at = 'end_of_transverse'


In [68]:
J = np.linspace(0, 1, 20) * 2e-8
phi = np.linspace(0, 2*np.pi, 20)

JJ, PP = np.meshgrid(J, phi)

xx = np.sqrt(2.*JJ*bx)*np.cos(PP)
xxp = -np.sqrt(2.*JJ/bx)*(np.sin(PP) + ax*np.cos(PP))
yy = np.sqrt(2.*JJ*by)*np.cos(PP)
yyp = -np.sqrt(2.*JJ/by)*(np.sin(PP) + ay*np.cos(PP))

In [69]:
bunch = sps.generate_6D_Gaussian_bunch(n_macroparticles=n_macroparticles, intensity=1e11,
                                       epsn_x=4.5e-6, epsn_y=4.5e-6, sigma_z=0.23)
# print bunch.sigma_x(), bunch.sigma_y()
# exit(-1)
bunch.update({
        'x': xx.flatten(),
        'xp': xxp.flatten(),
        'y': yy.flatten(),
        'yp': yyp.flatten()})


# =====================================
# Add octupoles
from PyHEADTAIL.multipoles.multipoles import ThinSextupole, ThinOctupole

# sex = ThinSextupole(k2l=2)
# sps.install_after_each_transverse_segment(sex)
oct = ThinOctupole(k3l=1000)
sps.install_after_each_transverse_segment(oct)


# ===============
# Do the tracking
for i in xrange(n_turns):
    for m in sps.one_turn_map:
        m.track(bunch)

    x[i,:] = bunch.x
    xp[i,:] = bunch.xp
    y[i,:] = bunch.y
    yp[i,:] = bunch.yp

    
# =======================
# Some frequency analysis
from PySussix import Sussix

tunesx = np.zeros(n_macroparticles)
tunesy = np.zeros(n_macroparticles)

ssx = Sussix()
ssx.sussix_inp(nt1=1, nt2=n_turns, idam=2, ir=0, tunex=0.2, tuney=0.2)

# fig = plt.figure(figsize=(20, 10))
# ax1 = fig.add_subplot(121)
# ax2 = fig.add_subplot(122)
# plt.ion()
for i in xrange(n_macroparticles):
    if i%20==0:
        sys.stdout.write("\r--> Processing particle {:d}".format(i))
    ssx.sussix(x[:,i], bx*xp[:,i], y[:,i], by*yp[:,i], x[:,i], xp[:,i])
    tunesx[i] = ssx.ox[np.argmax(ssx.ax)]
    tunesy[i] = ssx.oy[np.argmax(ssx.ay)]
    
#     ax1.stem(ssx.ox, ssx.ax/max(ssx.ax))
#     ax1.plot(tunesx[i], 1, 'o')
#     ax2.stem(ssx.oy, ssx.ay/max(ssx.ay))
#     ax2.plot(tunesy[i], 1, 'o')
#     plt.draw()
#     plt.pause(0.1)
#     ax1.cla()
#     ax2.cla()

plt.close('all')
# ===============
# Plot output
fig = plt.figure(figsize=(20, 10))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132, sharex=ax1, sharey=ax1)
ax3 = fig.add_subplot(133)

ax1.plot(x, xp, '.')
ax2.plot(y, yp, '.')
ax1.set_xlim(-1e-2, 1e-2)
ax1.set_ylim(-1e-4, 1e-4)

ax3.plot(tunesx, tunesy, '.')


plt.show()

--> Processing particle 380

In [17]:
# =====================================
# Input parameters
n_segments = 3
Qx, Qy = 20.20, 10.18

n_turns = 2048
n_sigmas = 6
n_macroparticles = 1000
sx, sy = 0.5e-3, 0.5e-3

# =====================================
# Build machine and bunch
x = np.zeros((n_turns, n_macroparticles))
xp = np.zeros((n_turns, n_macroparticles))
y = np.zeros((n_turns, n_macroparticles))
yp = np.zeros((n_turns, n_macroparticles))

sps = SPS(optics='', machine_configuration='Q20-flattop', n_segments=n_segments, accQ_x=Qx, accQ_y=Qy, RF_at='end_of_transverse')
sps.one_turn_map.pop(-1)
bx = sps.transverse_map.beta_x[0]
by = sps.transverse_map.beta_y[0]

bunch = sps.generate_6D_Gaussian_bunch(n_macroparticles=n_macroparticles, intensity=1e11,
                                       epsn_x=3.5e-6, epsn_y=3.5e-6, sigma_z=0.23)
# print bunch.sigma_x(), bunch.sigma_y()
# exit(-1)
# bunch.update({
#         'x': np.linspace(-n_sigmas*sx, n_sigmas*sx, n_macroparticles),
#         'xp': np.zeros(n_macroparticles),
#         'y': np.linspace(-n_sigmas*sx, n_sigmas*sx, n_macroparticles),
#         'yp': np.zeros(n_macroparticles)})


# =====================================
# Add octupoles
from PyHEADTAIL.multipoles.multipoles import ThinSextupole, ThinOctupole

# sex = ThinSextupole(k2l=2)
# sps.install_after_each_transverse_segment(sex)
oct = ThinOctupole(k3l=1000)
sps.install_after_each_transverse_segment(oct)


# ===============
# Do the tracking
for i in xrange(n_turns):
    for m in sps.one_turn_map:
        m.track(bunch)

    x[i,:] = bunch.x
    xp[i,:] = bunch.xp
    y[i,:] = bunch.y
    yp[i,:] = bunch.yp

    
# =======================
# Some frequency analysis
from PySussix import Sussix

tunesx = np.zeros(n_macroparticles)
tunesy = np.zeros(n_macroparticles)

ssx = Sussix()
ssx.sussix_inp(nt1=1, nt2=n_turns, idam=2, ir=0, tunex=0.2, tuney=0.2)

for i in xrange(n_macroparticles):
    if i%20==0:
        sys.stdout.write("\r--> Processing particle {:d}".format(i))
    ssx.sussix(x[:,i], bx*xp[:,i], y[:,i], by*yp[:,i], x[:,i], xp[:,i])
    tunesx[i] = ssx.ox[np.argmax(ssx.ax)]
    tunesy[i] = ssx.oy[np.argmax(ssx.ay)]


# ===============
# Plot output
fig = plt.figure(figsize=(20, 10))
ax1 = fig.add_subplot(131)
ax2 = fig.add_subplot(132, sharex=ax1, sharey=ax1)
ax3 = fig.add_subplot(133)

ax1.plot(x, xp, '.')
ax2.plot(y, yp, '.')
ax1.set_xlim(-1e-2, 1e-2)
ax1.set_ylim(-1e-4, 1e-4)

ax3.plot(tunesx, tunesy, '.')


plt.show()

Synchrotron init. From kwargs: n_segments = 3
Synchrotron init. From kwargs: accQ_y = 10.18
Synchrotron init. From kwargs: accQ_x = 20.2
Synchrotron init. From kwargs: optics = ''
Synchrotron init. From kwargs: RF_at = 'end_of_transverse'
--> Processing particle 240

KeyboardInterrupt: 

***

In [33]:
fig, axes = plt.subplots(1, figsize=(12,10))

# plt.ion()
for i in xrange(n_turns):
    for m in sps.one_turn_map:
        m.track(bunch)
        
    if i%10==0:
        plt.plot(bunch.x, bunch.xp, '.')
        axes.set_xlim(-1e-2, 1e-2)
        axes.set_ylim(-1e-4, 1e-4)
        plt.draw()
        plt.pause(0.1)
        axes.cla()

plt.show()